In [432]:
import numpy as np
import pandas as pd

In [433]:
hidden = []
def subset_sum(numbers, target, partial=[]):
    s = sum(partial)
    # check if the partial sum is equals to target
    if s == target:
       # print("sum(%s)=%s" % (partial, target))
        hidden.append(partial)
    if s >= target:
        return  # if we reach the number why bother to continue
    for i in range(len(numbers)):
        n = numbers[i]
        subset_sum(numbers[:i+1], target, partial + [n])

In [434]:
N = 36
subset_sum(range(2,37),N)

maxi = 0; mini = 1000
for i in range(len(hidden)):
    num = hidden[i]
    wnum = 10 * (num[0]-1)
    for j in range(len(num)-1):
        wnum += num[j]*(num[j+1]-1)
    wnum +=  num[len(num)-1]
    if maxi < wnum:
        maxi = wnum
        max_hidden = num
    if mini > wnum:
        mini = wnum
        min_hidden = num
print('最多情况: ', maxi, max_hidden, '\n最少情况: ', mini, min_hidden)

最多情况:  510 [22, 14] 
最少情况:  46 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [435]:
def sign(s):
    return np.where(s >= 0, 1, -1)

def dertanh(s):
    return 1-np.tanh(s)**2

In [461]:
def loadData(filename):
    data = pd.read_csv(filename, sep='\s+', header=None)
    data = data.as_matrix()
    col, row = data.shape
    X = np.c_[np.ones((col, 1)), data[:, 0: row-1]]
    Y = data[:, row-1: row]
    return X, Y

X_train, Y_train = loadData('hw4_nnet_train.dat')
Xtest, Ytest = loadData('hw4_nnet_test.dat')

/home/sean/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [467]:
def backprop(X, Y, eta, s, r, T):
    m, n = X.shape
    W1 = np.random.uniform(low=-r, high=r, size=(3, s))
    W2 = np.random.uniform(low=-r, high=r, size=(s + 1, 1))
    for i in np.arange(T):
        random_m = np.random.randint(0, m)
        x0 = X[random_m, :]
        y0 = Y[random_m][0]
        S1 = x0.dot(W1)
        X1 = np.tanh(S1)
        X1 = np.concatenate((np.ones((1, 1)), X1.reshape(1, len(X1))), axis=1)
        S2 = X1.dot(W2)
        X2 = np.tanh(S2)[0][0]
        delta2 = -2*(y0 - X2)
        delta1 = (delta2 * W2[1:, :]).T * dertanh(S1)
        W1 -= eta * np.mat(x0).T.dot(delta1)
        W2 -= eta * X1.T * delta2
    return W1, W2

In [468]:
def err_func(W1, W2, X, Y):
    row, col = X.shape
    x = X
    x = np.c_[np.ones((row, 1)), np.tanh(x.dot(W1))]
    Yhat = np.tanh(x.dot(W2))
    return np.sum(sign(Yhat) != Y)/row

In [473]:
def test_m():
    eout = np.inf
    repeats = 5
    M = [1, 6, 11, 16, 21]
    for p in M:
        eout_temp = 0
        for repeat in np.arange(repeats):
            W1, W2 = backprop(X_train, Y_train, 0.1, p, 0.1, 50000)
            error =  err_func(W1, W2, Xtest, Ytest)
            eout_temp += error
        eout_temp = eout_temp / repeats
        print("m = %d, error = %f" %(p, eout_temp))
        if eout > eout_temp:
            eout = eout_temp
            best_m = p
    return eout, best_m

In [474]:
test_m()

m = 1, error = 0.288800
m = 6, error = 0.036000
m = 11, error = 0.036000
m = 16, error = 0.036000
m = 21, error = 0.036800


(0.036, 6)

In [475]:
def test_r():
    eout = np.inf
    repeats = 5
    r = [0, 0.1, 10, 100, 1000]
    for p in r:
        eout_temp = 0
        for repeat in np.arange(repeats):
            W1, W2 = backprop(X_train, Y_train, 0.1, 3, p, 50000)
            error =  err_func(W1, W2, Xtest, Ytest)
            eout_temp += error
        eout_temp = eout_temp / repeats
        print("r = %f, error = %f" %(p, eout_temp))
        if eout > eout_temp:
            eout = eout_temp
            best_r = p
    return eout, best_r

In [476]:
test_r()

m = 0, error = 0.505600
m = 0, error = 0.036000
m = 10, error = 0.151200
m = 100, error = 0.340000
m = 1000, error = 0.475200


(0.036, 0.1)

In [479]:
def test_eta():
    eout = np.inf
    repeats = 5
    eta = [0.001, 0.01, 0.1, 1, 10]
    for p in eta:
        eout_temp = 0
        for repeat in np.arange(repeats):
            W1, W2 = backprop(X_train, Y_train, p, 3, 0.1, 50000)
            error =  err_func(W1, W2, Xtest, Ytest)
            eout_temp += error
        eout_temp = eout_temp / repeats
        print("eta = %f, error = %f" %(p, eout_temp))
        if eout > eout_temp:
            eout = eout_temp
            best_eta = p
    return eout, best_eta

In [480]:
test_eta()

eta = 0.001000, error = 0.128800
eta = 0.010000, error = 0.036000
eta = 0.100000, error = 0.036000
eta = 1.000000, error = 0.294400
eta = 10.000000, error = 0.484800


(0.036, 0.01)